# Add Deconvolution Pre-Stem to ResNet50

### Background

The ResNet50 architecture does not learn well (or at all) with small image sizes, such as the CIFAR-10 and CIFAR-100 whose image size is 32x32. The reason is that the feature maps are downsampled too soon in the architecture and become 1x1 (single pixel) before reaching the bottleneck layer prior to the classifier.

The ResNet50 was designed for 224 x 224 but will work well for size 128 x 128.

### Solution

We could updsample the CIFAR-10 images upstream from 32 x 32 to 128 x 128, using an interpolation algorithm such as BI-CUBIC --but this 'hardwired' interpolation may not be the best and may introduce artifacts. Additionally, being upstream from the model, it is generally an inefficient method.

Instead, we will add a Pre-Stem Group at the bottom (input) layer of a stock ResNet to learn the best upsampling using deconvolution. Additionally, the pre-stem becomes part of the graph.

### Step 1

We start with a stock `ResNet50` without a classifier and reset the input shape to (128, 128, 3), which we name as the `base` model.

Next, we add the classifier layer as a Dense layer of 10 nodes, which we name as the `resnet` model.

In [1]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Conv2DTranspose

base = ResNet50(include_top=False, input_shape=(128, 128, 3), pooling='max')
outputs = Dense(10, activation='softmax')(base.output)
resnet = Model(base.input, outputs)
resnet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 134, 134, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 64, 64, 64)   256         conv1[0][0]                      
______________________________________________________________________________________________

### Step 2

We construct a pre-stem group using two deconvolutions (also called a transpose convolution):

    1. First deconvolution takes (32, 32, 3) as input and upsamples to (64, 64, 3).
    2. Second deconvolution upsamples to (128, 128, 3)
    3. We use the add() method to attach the pre-stem to the resnet model.
    
Essentially, the pre-stem takes the (32, 32, 3) CIFAR-10 inputs and outputs (128, 128, 3) which is then the input to the resnet model.

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu', input_shape=(32,32,3)))
model.add(BatchNormalization())
model.add(Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(resnet)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_4 (Conv2DTr (None, 64, 64, 3)         84        
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 128, 128, 3)       84        
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
model (Model)                (None, 10)                23608202  
Total params: 23,608,394
Trainable params: 23,555,262
Non-trainable params: 53,132
_________________________________________________________________


### Train the Model

Let's partially train the model to demonstrate how a pre-stem works. First, for ResNet50 I find a reliable choice of optimizer and learning rate is the Adam optimizer with a learning rate = 0.001. While the batch normalization should provide the ability for higher learning rates, I find with higher ones on ResNet50 it per epoch loss does not converge.

We will use the CIFAR-10 builtin dataset and normalize the image data and one-hot encode the labels upstream from the model.

We will then use the fit() method for a small number of epochs (5) and set aside 10% of the training data for the per epoch validation data.

From my test run, I got:

    Epoch 1: 27.7%
    Epoch 2: 33.8%
    Epoch 3: 42.9%
    Epoch 4:
    Epoch 5:

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = (x_train / 255.0).astype(np.float32)
x_test  = (x_test  / 255.0).astype(np.float32)

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

pivot=int(len(x_train) * 0.1)
x_val = x_train[:pivot]
y_val = y_train[:pivot]
x_train = x_train[pivot:]
y_train = y_train[pivot:]

print(x_train.shape)

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=1, validation_split=0.1)

Train on 40500 samples, validate on 4500 samples
Epoch 1/5
40500/40500 [==============================] - 1956s 48ms/sample - loss: 2.3762 - acc: 0.2659 - val_loss: 2.7859 - val_acc: 0.2771
Epoch 2/5
40500/40500 [==============================] - 1951s 48ms/sample - loss: 2.1494 - acc: 0.3531 - val_loss: 2.4331 - val_acc: 0.3387
Epoch 3/5
40500/40500 [==============================] - 1946s 48ms/sample - loss: 2.0251 - acc: 0.4025 - val_loss: 1.5739 - val_acc: 0.4291
Epoch 4/5
  736/40500 [..............................] - ETA: 31:44 - loss: 1.8896 - acc: 0.4647